# The Freva command line interface

This notebook introduces the usage of the Freva command line interface - cli. The tutorial assumes that you have already access to Freva either because you’ve setup an instance yourself or one has been setup by the Freva admin team. Hence it is assumed that you know how to access Freva. If this is not the case please contact one of your Freva admins for help.

A general usage overview of the available Freva sub-commands is available via the --help option:

In [ ]:
freva --help

The most common sub-commands are `databrowser` and `plugin`. You can get more help on the actual commands using the sub-commands `--help` option, for example getting help on the databrowser usage:

In [ ]:
freva databrowser --help

__Note__: Instead of using sub-commands you have also the option to use commands. For example the command `freva-databrowser` is equivalent to `freva databrowser`, `freva-user-data` to `freva user-data` etc.

## Searching for data: the freva-databrowser command

All files available on in the project are scanned and indexed via a data search server. This allows you to query the server which responds almost immediately. To search for data you can either use the `freva-databrowser` command or the `freva databrowser` sub-command. Let’s inspect the help menu of the databrowser sub-command:

In [ ]:
freva-databrowser --help

The databrowser expects a list of `key=value` pairs. The order of the pairs doesn’t really matter. Most important is that you don’t need to split the search according to the type of data you are searching for. You can search for any files, for example observations, reanalysis and model data at the same time. Also important is that all queries are case insensitive. You can also search for attributes themselves instead of file paths. For example you can search for the list of variables available that satisfies a certain constraint (e.g. sampled 6hr, from a certain model, etc).

In [ ]:
freva-databrowser project=observations variable=pr model='cp*'

In some cases it might be useful to know how many files are found in the `databrowser` for certain search constraints. In such cases you can use the `count` flag to count the number of found files instead of getting the files themselves.

In [ ]:
freva-databrowser project=observations --count

Sometimes it might be useful to subset the data you’re interested in by time. To do so you can use the time search key to subset time steps and whole time ranges. For example let’s get the for certain time range:

In [ ]:
freva-databrowser project=observations time='2016-09-02T22:15 to 2016-10'

The default method for selecting time periods is `flexible`, which means all files are selected that cover at least start or end date. The `strict` method implies that the entire search time period has to be covered by the files. Using the `strict` method in the example above would only yield on file because the first file contains time steps prior to the start of the time period:

In [ ]:
freva-databrowser project=observations time='2016-09-02T22:15 to 2016-10' --time-select strict

You might as well want to know about possible values that an attribute can take after a certain search is done. For this you use the `--facet` flag (facets are the attributes used to search for and sub set the data). For example to see all `facets` that are available in the observations project:

In [ ]:
freva-databrowser project=observations --all-facets

In [ ]:
freva-databrowser --facet time_frequency --facet variable project=observations

You can also retrieve information on how many facets are found by the databrowser by giving the `count` flag

In [ ]:
freva-databrowser --facet time_frequency --facet variable project=observations --count

# Running data analysis plugins: the freva-plugin command

Already defined data analysis tools can be started with the `freva-plugin` command or the `freva plugin` sub-command. Let’s inspect the help menu of the plugin command:

In [ ]:
freva-plugin --help

As the help menu suggests you can list all available tools using the `-l` option (or `--list`, `--list-tools`):

In [ ]:
freva-plugin -l

This means currently we have two plugins available (animator and dummyplugin). The general syntax is freva-plugin `<plugin-name> [options]` for example to inspect the documentation of a certain plugin you can use the `--doc` option. Here we concentrate on the Animator plugin. A simple plugin that creates animations of geospatial data. The basic usage of that command can be retrieved by:

In [ ]:
freva-plugin --doc animator

The parameters are also given as `key=values` pairs. But not all of the above parameters are mandatory. Let’s use one project search key and animate its content.

In [ ]:
freva plugin animator project=observations variable=pr cmap=Blues fps=5 output_unit=mm/h vmin=0 vmax=5 suffix=gif

This plugin will run in so called interactive mode. That means that it will run on the login node and block your shell until the command is completed. This can be problematic if you have jobs that might take time to finish. An alternative is setting the `-–batchmode` flag. This flag tells the plugin to submit a job to the computing queue. The computing nodes are the core of any high performance computing system. Let’s submit the previous plugin job to the computing queue:

In [ ]:
freva plugin animator project=observations variable=pr cmap=Blues fps=5 output_unit=mm/h vmin=0 vmax=5 suffix=gif --batchmode

# Inspecting previous analysis jobs: the freva-history command

Sometimes it can be useful to access the status of past plugin applications. The `freva-history` command or `freva history` sub-command can do that:

In [ ]:
freva-history --limit 1 --plugin dummyplugin

In [ ]:
freva-history --entry-ids 1 --full-text

To re-run a command of a past configuration you can use the --return-command option to get the command that was used:

In [ ]:
freva-history  --entry-ids 136 --return-command

# Managing your own datasets: the freva-user-data command

Freva offers the possibility to share custom datasets with other users by making it searchable via `freva-databrowser`. With help of the `freva-user-data` command users can add their own data to the central data location, (re)-index or delete data in the databrowser.

__Note__: Any data that has been added by users will be assigned a special project name: `project=user-$USER`.
Let’s inspect the help menu of the freva-user-data or freva user-data command:

In [ ]:
freva-user-data --help

## Add new data to the databrowser

To be able to add data to the databrowser the file names must follow a strict standard and the files must reside in a specific location. The add sub command takes care about the correct file naming and location. No pre requirements other than the file has to be a valid netCDF or grib file are assumed. In other words this sub command places the user data with the correct naming structure to the correct location.

In [ ]:
freva-user-data add --help

Suppose you’ve gotten data from somewhere and want to add this data into the databrowser to make it accessible to others. In this specific example we assume that you have stored your original data in the `/tmp/my_awesome_data` folder. E.g `/tmp/my_awesome_data/outfile_0.nc...tmp/my_awesome_data/outfile_9.nc` The routine will try to gather all necessary metadata from the files. You’ll have to provide additional metadata if mandatory keywords are missing. To make the routine work in this example we have to provide the institute, model and experiment keywords:



In [ ]:
freva-user-data add eur-11b /tmp/my_awesome_data/outfile_?.nc \
--institute clex --model UM-RA2T --experiment Bias-correct
freva-databrowser experiment=bias-correct

## Remove your data from the databrowser

The delete sub command removes entries from the databrowser and if necessary existing files from the central user data location.

In [ ]:
freva-user-data delete --help

Any data in the central user directory that belongs to the user can be deleted from the databrowser and also from the central data location:

In [ ]:
freva-user-data delete /tmp/user_data/user-$USER
freva-databrowser experiment=bias-correct

## (Re)-Index existing data to the databrowser

The index subcommand can be used to update the databrowser for existing user data. For example, if data has been removed from the databrowser it can be re-added:

In [ ]:
freva-user-data index --help

In [ ]:
freva-user-data delete /tmp/user_data/user-$USER
freva-databrowser experiment=bias-correct
